<a href="https://colab.research.google.com/github/ampnb/EEG-Mind-Wandering/blob/main/MW_recording_Future.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import os
import mne
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from psychopy import visual, core

def main():
    BOARD_ID = BoardIds.UNICORN_BOARD.value
    RECORDING_DIR = "./"
    PARTICIPANT_ID = "participant_001"
    CONDITION_ORDER = ['PAST+', 'PAST-', 'FUTURE+', 'FUTURE-']

    BoardShim.enable_dev_board_logger()
    params = BrainFlowInputParams()
    board = BoardShim(BOARD_ID, params)
    board.prepare_session()
    board.start_stream()

    win = visual.Window(size=(800, 600), fullscr=False, units='norm', color=[-1, -1, -1])

    # Counterbalance condition order
    random.shuffle(CONDITION_ORDER)

    for condition in CONDITION_ORDER:
        # Present cue for 15 seconds
        cue = visual.TextStim(win, text=condition, color='white', height=0.1)
        cue.draw()
        win.flip()
        time.sleep(15)

        # Record EEG data during the 5-minute mind-wandering period
        start_time = time.time()
        while time.time() - start_time < 300:
            pass  # Replace this with a call to the function that records data for the given period (e.g., board.get_board_data())

        # Present Likert scale for 30 seconds
        likert = visual.TextStim(win, text="Rate the emotional valence of your thoughts (-5 to +5)", color='white', height=0.1)
        likert.draw()
        win.flip()
        time.sleep(30)

        # Rest period for 60 seconds
        win.flip()
        time.sleep(60)

    # Stop recording and save data
    data = board.get_board_data()
    board.stop_stream()
    board.release_session()
    win.close()

    # Process and save the data
    raw = getdata(data, BOARD_ID)
    save_raw(raw, RECORDING_DIR, PARTICIPANT_ID)

def getdata(data, board_id):
    eeg_channels = BoardShim.get_eeg_channels(board_id)
    data = data[eeg_channels, :]
    ch_names = BoardShim.get_eeg_names(board_id)
    ch_types = ['eeg'] * len(eeg_channels)
    sfreq = BoardShim.get_sampling_rate(board_id)

    info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)
    raw = mne.io.RawArray(data, info)
    montage = mne.channels.make_standard_montage('standard_1020')
    raw.set_montage(montage)

    return raw

def save_raw(raw, dir, participant_id):
    folder_path = create_session_folder(participant_id, dir)
    raw.save(os.path.join(folder_path, f'{participant_id}_emotion_mind_wandering_raw.fif'), overwrite=True)
    return os.path.basename(folder_path)

def create_session_folder(subj, dir):
    base_path = os.getcwd() + "/"
    dir = base_path + dir
    folder_name = f'{subj}'
    if os.path.isdir(os.path.join(dir, folder_name)):
        folder_path = os.path.join(dir, folder_name)
    else:
        folder_path = os.path.join